Be prepared for failure and retries:
- llms don't always do as their told (for example json might be perfect)
- llms services might not be available perfectly (like OpenAI)

In [1]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

We can use a parser to check the output of the llm and we can also use it to provide hints to the llm to improve it's output.

In [3]:
# https://python.langchain.com/docs/modules/model_io/output_parsers/retry
# https://python.langchain.com/docs/modules/model_io/output_parsers/structured

from langchain.prompts import (
    PromptTemplate,
)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
    RetryOutputParser,
)
from pydantic import BaseModel, Field, validator
from typing import List


For example we specify the schema of the output we expected

In [4]:

template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""

class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

Here's an example of something that would be bad as we parse it.
It misses one of the two fields in the response.

In [6]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

bad_response = '{"action": "search"}'
parser.parse(bad_response)



ValidationError: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

Now we give the llm hints through the parse instructions we described.
See how it uses that under the hood.

In [ ]:
import sys
sys.path.append('../developer')

from _lessonshelper.pretty_print_callback_handler import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()


fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(callbacks=[callback]))
fix_parser.parse(bad_response)


[chat_model][start] - prompts : Instructions:
--------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"action": {"title": "Action", "description": "action to take", "type": "string"}, "action_input": {"title": "Action Input", "description": "input to the action", "type": "string"}}, "required": ["action", "action_input"]}
```
--------------
Completion:
--------------
{"action": "search"}
--------------

Above, the Completion did not satisfy the constraints given in the Instructions.
Error:
--------------
OutputParserException('Failed to parse Action from completion {

Action(action='search', action_input='query')

Now that we have the instructions, we can have it retry that automatically if it doesn't comply.

In [ ]:
from langchain.output_parsers import RetryWithErrorOutputParser

retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0, callbacks=[callback])
)

retry_parser.parse_with_prompt(bad_response, prompt_value)




[llm][start] - prompts: Prompt:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"action": {"title": "Action", "description": "action to take", "type": "string"}, "action_input": {"title": "Action Input", "description": "input to the action", "type": "string"}}, "required": ["action", "action_input"]}
```
who is leo di caprios gf?

Completion:
{"action": "search"}

Above, the Completion did not satisfy the constraints given in the Prompt.
Details: OutputParserException('Failed to parse Action from completion {"action": "search"}. Got: 1 validation err

Action(action='search', action_input='who is leo di caprios gf?')